<a href="https://colab.research.google.com/github/jlgjosue/ia-dl-unyleya/blob/master/Tarefa_8_Exemplos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p>Leia o primeiro capítulo da Unidade 4 e replique os exemplos apresentados, analisando-o linha a linha e comentando a função de cada comando presente no código. Comente cada saída gerada, justificando-a.
<p>Você deverá enviar o código em Python.

In [0]:
#Inicialmente, vamos importar as bibliotecas:
import torch

import torch.nn as nn

import torch.optim as optim

import torchvision
from torchvision import models, transforms

#Em seguida, vamos definir o tamanho do lote (batch_size):

batch_size = 50

In [5]:
# Para definir o conjunto de dados de treinamento, vamos levar em consideração que as imagens CIFAR-10 têm tamanho 32x32, enquanto as redes ImageNet necessitam de dados de tamanho 224x224. Então, vamos aumentar as imagens do CIFAR-10 de 32x32 para 224x224. É necessário padronizar os dados do CIFAR-10 utilizando a média e o desvio-padrão da ImageNet, e é necessário adicionar um pequeno aumento de dados (flip):

# Dados de Treinamento

train_data_transform = transforms.Compose([

     transforms.Resize(224),

     transforms.RandomHorizontalFlip(),

     transforms.RandomVerticalFlip(),

     transforms.ToTensor(),

     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

])

 

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_data_transform)
 

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

# Realizar os seguintes passos com os dados de validação e teste:

val_data_transform = transforms.Compose([

     transforms.Resize(224),

     transforms.ToTensor(),

     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

])

 

val_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=val_data_transform) 

val_order = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)

# Escolher um dispositivo, preferencialmente GPU:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Definir o modelo de treinamento. Ao contrário do Keras, com PyTorch é necessário realizar as repetições manualmente sobre os dados de treinamento. Este método repete uma vez por todo o conjunto de treinamento (uma iteração) e aplica um otimizador após cada passo:

def train_model(model, loss_function, optimizer, data_loader):
     # Definindo o modelo para o modo de treinamento
     model.train()
     current_loss = 0.0
     current_acc = 0
     # Repete sobre o conjunto de treinamento

     for i, (inputs, labels) in enumerate(data_loader):

         # Envia a entrada ou os rótulos para a GPU

         inputs = inputs.to(device)

         labels = labels.to(device)

 

         # reinicia os parâmetros do gradiente

         optimizer.zero_grad()

 

         with torch.set_grad_enabled(True):

              # Passo adiante

              outputs = model(inputs)

              _, predictions = torch.max(outputs, 1)

              loss = loss_function(outputs, labels)

 

              # passo para trás

              loss.backward()

              optimizer.step()

 

         # Estatística

         current_loss += loss.item() * inputs.size(0)

         current_acc += torch.sum(predictions == labels.data)

 

     total_loss = current_loss / len(data_loader.dataset)

     total_acc = current_acc.double() / len(data_loader.dataset)

 

     print('Train Loss: {:.4f}; Accuracy: {:.4f}'.format(total_loss, total_acc))

# Definir o primeiro cenário de transferência de aprendizado, em que a rede pré-treinada é utilizada como extratora de características. Será utilizada uma rede popular, chamada ResNet-18, e o PyTorch fará o download dos pesos pré-treinados automaticamente. Vamos substituir a última camada da rede por uma nova camada com 10 classes de saída (uma para cada classe do CIFAR-10). Vamos eliminar as camadas da rede original do passo para trás, e atualizar os pesos apenas da camada adicionada utilizando o otimizador ADAM. Vamos rodar o treinamento e verificar a acurácia da rede a cada iteração:

def tl_feature_extractor(epochs=3):

     # Carregar o modelo pré-treinado

     model = torchvision.models.resnet18(pretrained=True)

 

     # Excluir os parâmetros existentes do passo para trás

     for param in model.parameters():

         param.requires_grad = False

 

     # As novas camadas require, por padrão,

     # requires_grad=True

     num_features = model.fc.in_features

     model.fc = nn.Linear(num_features, 10)

 

     # Transferindo para a GPU (se disponível)

     model = model.to(device) 

     loss_function = nn.CrossEntropyLoss()
 

     # Otimizar apenas os parâmetros da última camada

     optimizer = optim.Adam(model.fc.parameters())
 

     # treinamento

     for epoch in range(epochs):

         print('Epoch {}/{}'.format(epoch + 1, epochs)) 

         train_model(model, loss_function, optimizer, train_loader)

         test_model(model, loss_function, val_order)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
# Definir o segundo cenário para refinamento da rede original. Similar ao cenário anterior, porém, toda a rede será treinada novamente:

def tl_fine_tuning(epochs=3):
     # Carregar o modelo pré-treinado
     model = models.resnet18(pretrained=True)

     # Substituir a última camada
     num_features = model.fc.in_features
     model.fc = nn.Linear(num_features, 10) 

     # Transferir o modelo para a GPU

     model = model.to(device) 

     # Função de Perda

     loss_function = nn.CrossEntropyLoss() 

     # Otimização de todos os parâmetros

     optimizer = optim.Adam(model.parameters()) 

     # treinamento

     for epoch in range(epochs):

         print('Epoch {}/{}'.format(epoch + 1, epochs)) 

         train_model(model, loss_function, optimizer, train_loader)

         test_model(model, loss_function, val_order)

É possível rodar a rede de acordo com o cenário escolhido. Para o primeiro cenário, chamar a função tl_feature_extractor(epochs=5) e para o segundo cenário, chamar a função tl_fine_tuning(epochs=5). Para o primeiro cenário, a acurácia obtida foi de 76%, enquanto, no segundo cenário, a acurácia foi de 87%.